In [6]:
from collections import defaultdict
import matplotlib.pyplot as plt
from pathlib import Path
import json
import pickle
import numpy as np

with open("captions_train2014.json", "r") as read_file:
    captions_train = json.load(read_file)
    
with open("resnet18_features.pkl", "rb") as read_file:
    images_dict = pickle.load(read_file)

keys_array = np.array(list(images_dict.keys()))
        
captions_dict = defaultdict(list)
captions_list = list()
for annotation in captions_train['annotations']:
    if annotation['image_id'] in keys_array:
        captions_dict[annotation['image_id']].append(annotation)
        captions_list.append(annotation['caption'])

with open('images_dict.pkl', 'wb') as handle:
        pickle.dump(images_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('captions_dict.pkl', 'wb') as handle:
        pickle.dump(captions_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('captions_list.pkl', 'wb') as handle:
        pickle.dump(captions_list, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('keys_array.pkl', 'wb') as handle:
        pickle.dump(keys_array, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('captions_train.pkl', 'wb') as handle:
        pickle.dump(captions_train, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [72]:
with open("captions_train2014.json", "r") as read_file:
    captions_train = json.load(read_file)
for annotation in captions_train['annotations']:
    if annotation['image_id'] in keys_array:
        captions_dict[annotation['image_id']].append(annotation)
with open('captions_dict.pkl', 'wb') as handle:
        pickle.dump(captions_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [31]:
import re, string
def strip_punc(corpus: str):
    """ Removes all punctuation from a string.

        Parameters
        ----------
        corpus : str

        Returns
        -------
        str
            the corpus with all punctuation removed"""

    punc_regex = re.compile('[{}]'.format(re.escape(string.punctuation)))
    return punc_regex.sub("", corpus)

In [2]:
from pathlib import Path
import pickle
with open("./stopwords.txt", 'r') as r:
    stops = []
    for line in r:
        stops += [i.strip() for i in line.split('\t')]
        
with open('stopwords.pkl', 'wb') as handle:
        pickle.dump(stops, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [33]:
import numpy as np
vocabulary = defaultdict(int)
vocabulary_idf = dict()
for caption in captions_list:
    for word in strip_punc(caption).lower().split():
        if word not in stops and word in glove:
            vocabulary[word]+=1
for i in vocabulary.keys():
    vocabulary_idf[i] = np.log10(len(captions_list)/vocabulary[i])
    


In [49]:
with open('vocabulary.pkl', 'wb') as handle:
        pickle.dump(vocabulary, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
with open('vocabulary_idf.pkl', 'wb') as handle:
        pickle.dump(vocabulary_idf, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
with open('tags_dict.pkl', 'wb') as handle:
        pickle.dump(tags_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
with open('truth_tags.pkl', 'wb') as handle:
        pickle.dump(truth_tags, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [45]:
from collections import Counter
tags_dict = defaultdict(list)
truth_tags = dict()
for i,v in captions_dict.items():
    for annotation in v:
        for word in strip_punc(annotation['caption']).lower().split():
            if word in vocabulary and word in glove:
                tags_dict[i].append(word)

In [79]:
for i in keys_array:
    c = Counter(tags_dict[i])
    for tag, count in c.items():
        c[tag] = c[tag]**2
        c[tag] *= vocabulary_idf[tag]
    d = c.most_common(3)
    truth_tags[i] = np.zeros((3,50))
    for u in range(3):
        truth_tags[i][u] = glove[d[u][0]]

In [80]:
# from numba import njit

# @njit
def permutate(x):
    perms = [np.array([0,1,2]), np.array([0,2,1]), np.array([1,0,2]), np.array([1,2,0]), np.array([2,0,1]), np.array([2,1,0])]
    m,n = x.shape
    perm_array = np.zeros((6,m*n))
    for i in range(6):
        perm_array[i] = x[perms[i]].flatten()
    return perm_array

In [81]:
for i in truth_tags.keys():
    truth_tags[i] = permutate(truth_tags[i])
    y = np.linalg.norm(truth_tags[i], axis = 1).reshape(-1,1)
    truth_tags[i]/=y

In [ ]:
from gensim.models.keyedvectors import KeyedVectors as KV
glove = KV.load_word2vec_format('./glove.6B.50d.txt.w2v', binary=False)

In [82]:
with open('truth_tags.pkl', 'wb') as handle:
        pickle.dump(truth_tags, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [71]:
captions_dict[57870]

[{'image_id': 57870,
  'id': 787980,
  'caption': 'A restaurant has modern wooden tables and chairs.'},
 {'image_id': 57870,
  'id': 789366,
  'caption': 'A long restaurant table with rattan rounded back chairs.'},
 {'image_id': 57870,
  'id': 789888,
  'caption': 'a long table with a plant on top of it surrounded with wooden chairs '},
 {'image_id': 57870,
  'id': 791316,
  'caption': 'A long table with a flower arrangement in the middle for meetings'},
 {'image_id': 57870,
  'id': 794853,
  'caption': 'A table is adorned with wooden chairs with blue accents.'}]